In [14]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Tuple

In [15]:
class Perceptron:
    def __init__(self, inputs: int, f_activacion: Callable[[float], float]):
        """
        Parametros
        ----------
        inputs : `int`
            Numero de entradas
        f_activacion : `Callable[[float], float]`
            Función de activación
        """
        self.inputs = inputs
        self.f_activacion = f_activacion

        # Inicialización de los pesos
        # Se agrega un elemento para representar el bias
        # y se resta 0.5 para que quede entre -0.5 y 0.5
        self.w = np.random.rand(inputs + 1) - 0.5
    
    def eval(self, patron: np.ndarray) -> Tuple[np.ndarray, float, float]:
        """Evalua un patron y calcula el error como la diferencia entre
        el valor deseado y el calculado

        Parametros
        ----------
        patron : `ndarray`
            Vector de entrada

        Return
        -------
        `tuple`
            [Salida calculada, Error, Patron evaluado (con bias)]
        """
        x = np.hstack((-1, patron[:self.inputs]))
        # Producto interno
        z = np.inner(x, self.w)
        # No linealidad
        y = self.f_activacion(z)
        # Error
        err = patron[-1] - y
        return (y, err, x)

    def train(self, patron: np.ndarray, alpha: float) \
        -> Tuple[np.ndarray, float, float]:
        """Evalua un patron y a partir del error ajusta los pesos en base a la
        tasa de aprendizaje alpha

        Parametros
        ----------
        patron : `ndarray`
            Vector de entrada
        alpha : `float`
            Tasa de aprendizaje

        Return
        -------
        `tuple`
            [Nuevos pesos, Salida calculada, Error]
        """
        # Evaluación
        (y, err, x) = self.eval(patron)
        # Actualización de pesos
        self.w = self.w + (alpha * err) * x
        return (self.w, y, err)

In [16]:
# Función de activación "signo"
def f_sign(a: np.number | np.ndarray) -> np.number | np.ndarray:
    """Función signo: Si `a < 0` devuelve `-1`, en caso contrario devuelve `1`
    (vectorizable)
    """
    return (np.heaviside(a, 1) * 2) - 1

# Ejercicio 2

Realice un programa que permita generar un conjunto de particiones de entrenamiento y prueba a partir de un único archivo de datos en formato texto separado por comas. El programa debe permitir seleccionar la cantidad de particiones y el porcentaje de patrones de entrenamiento y prueba. Para probarlo:

1. El archivo `spheres1d10.csv` contiene una serie de datos generados a partir de los valores de la **Tabla 1**, con pequeñas desviaciones aleatorias ($< 10 \%$) en torno a ellos (**Figura 2(a)**). Realice con estos datos la validación cruzada del perceptrón simple con 5 particiones de entrenamiento y prueba con relación 80/20.

2. A partir de la misma tabla del ejemplo anterior, pero modificando el punto $x = [−1 + 1 − 1] \to y_d = 1$, se ha generado un conjunto de datos diferente. Los archivos `spheres2d10.csv`, `spheres2d50.csv` y `spheres2d70.csv` contienen los datos con desviaciones aleatorias de $10$, $50$ y $70 \%$, respectivamente (**Figuras 2(b)**, **2(c)** y **2(d)**). Realice la validación cruzada del perceptrón simple con 10 particiones de entrenamiento y prueba, con relación $80/20$

In [17]:
def create_partitions(filename: str, partitions: int, tr_rate: float, \
    CSV_delimiter : str = ',' ) -> Tuple[np.ndarray, np.ndarray] :
    """Crear particiones de entrenamiento y prueba
    
    A partir de un archivo CSV, genera un número dado de particiones
    con una distribución de patrones de entrenamiento y prueba según el
    `tr_rate` (en porcentaje). Se devuelven los patrones importados y 
    los índices correspondientes a los patrones que conforman cada una
    de las particiones.

    Parametros
    ----------
    filename : `str`
        Nombre del archivo CSV
    partitions : `int`
        Cantidad de particiones
    tr_rate : `float`
        Porcentaje de patrones de entrenamiento

    Return
    -------
    tuple
        [Patrones, Partición de entrenamiento, Partición de prueba]
    """
    # Cargar el archivo CSV
    patterns = np.genfromtxt(filename, delimiter=CSV_delimiter)
    num_patterns_ = patterns.shape[0]

    # Se genera un listado de indices para cada partición mezclados en orden
    # aleatorio
    partitions_idx = np.array([ \
        np.random.choice(range(num_patterns_), num_patterns_, replace=False)\
            for i in range(partitions)])

    # Se calcula el índice del corte entre patrones de train y test
    cut_ = int(np.ceil(num_patterns_ * tr_rate))

    # Al listado de indices para cada partición, se divide en el corte
    partitions_splitted = np.split(partitions_idx, [cut_], axis=1)

    # Se divide el dataset en train y test tomando los patrones según los índices
    # de cada partición
    # train = np.array([patterns[i,:] for i in partitions_splitted[0]])
    # test = np.array([patterns[i,:] for i in partitions_splitted[1]])

    return (patterns, partitions_splitted[0], partitions_splitted[1])


In [18]:
partitions = 5          # Cantidad de particiones
partition_rate = 0.8    # Porcentaje de patrones para train
(patterns, idx_train, idx_test) = \
    create_partitions('icgtp1datos/spheres1d10.csv', partitions, partition_rate)

In [26]:
# Cantidad de patrones de train
train_patterns = idx_train.shape[1]
# Cantidad de patrones de test
# test_patterns = idx_test.shape[1]

# Cantidad de entradas
inputs_number = patterns.shape[1]-1

N = 20          # Nro de épocas
alpha = 1E-4    # Tasa de aprendizaje
nu = 0.1        # Umbral de error

# Para calcular el error entre épocas se utilizará 10% de los patrones
num_patterns_val = int(np.floor(train_patterns * 0.1))

# Almacenar la tasa de error con datos de test de cada partición
tst_error_rate = np.ndarray((partitions))

for j in range(partitions):  # Para cada partición
    # "Separar" los patrones de entrenamiento de la partición
    partition = patterns[idx_train[j]]

    # Iniciar el perceptrón simple
    perceptron = Perceptron(inputs_number, f_sign)

    # Entrenamiento por épocas
    for i in range(N):                          # Para cada época
        for pattern in partition:               # Para todos los patrones
            perceptron.train(pattern, alpha)    # Entrenar
    
        # Para cada época se calcula el error
        # Obtener índices aleatorios de los patrones que se utilizaran
        # para la validación
        val_patterns_idx = np.random.choice(range(partition.shape[0]), num_patterns_val, replace=False)
        
        errors = 0  # Acumulador de errores
        for pattern in partition[val_patterns_idx]:  # Para cada patrón de validación
            (y, _, _) = perceptron.eval(pattern)     # Evaluar
            errors += int(y != pattern[-1])          # Contar si es un error o no
        err_rate = errors / num_patterns_val         # Calcular la tasa de error

        # Evolución del error por época
        # print(f'Partición {j} - Época {i}: {err_rate * 100:.2f} %')

        # Si el error calculado es menos del umbral cortar
        if (err_rate < nu):
            break

    print(f'Partición {j}: Total de épocas = {i} - Error = {err_rate * 100:.2f} %')
    
    # Evaluar el perceptron con la partición de prueba
    errors = 0  # Acumulador de errores
    for patron in patterns[idx_test[j]]:        # Para cada patrón de la partición de test
        (y, _, _) = perceptron.eval(patron)     # Evaluar
        errors += int(y != patron[-1])          # Contar si es un error o no
    err_rate = errors / num_patterns_val        # Calcular el error medio
    tst_error_rate[j] = err_rate

    print(f'Partición {j}: Error con datos de prueba {err_rate * 100:.2f} %')
    

Partición 0: Total de épocas = 19 - Error = 18.75 %
Partición 0: Error con datos de prueba 58.75 %
Partición 1: Total de épocas = 19 - Error = 8.75 %
Partición 1: Error con datos de prueba 22.50 %
Partición 2: Total de épocas = 19 - Error = 12.50 %
Partición 2: Error con datos de prueba 28.75 %
Partición 3: Total de épocas = 19 - Error = 17.50 %
Partición 3: Error con datos de prueba 50.00 %
Partición 4: Total de épocas = 19 - Error = 8.75 %
Partición 4: Error con datos de prueba 20.00 %
